In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pmdarima as pm
import numpy as np
from statsmodels.tsa.stattools import adfuller

In [ ]:
import pandas as pd
import boto3
import time 
n_periodos_a_predecir = 5

athena_client = boto3.client('athena', region_name='us-east-1')
query = "SELECT * FROM nacimientos_colombia_limpio;"
# Start Athena query
response = athena_client.start_query_execution(
    QueryString=query,
    QueryExecutionContext={'Database': 'tfm-educ-app-silver'},
    WorkGroup = 'tfm-educ-app-athena'
)
query_execution_id = response['QueryExecutionId']

while True:
    status = athena_client.get_query_execution(QueryExecutionId=query_execution_id)['QueryExecution']['Status']['State']
    if status in ['SUCCEEDED', 'FAILED']:
        break
    time.sleep(2)
if status == 'SUCCEEDED':
    # Get query results
    result_response = athena_client.get_query_results(QueryExecutionId=query_execution_id)
    data = []
    columns = None
    next_token = None
    
    while True:
        if columns is None:
            columns = [col["VarCharValue"] for col in result_response["ResultSet"]["Rows"][0]["Data"]]
        if next_token:
            result_response = athena_client.get_query_results(QueryExecutionId=query_execution_id, NextToken=next_token)
        for row in result_response["ResultSet"]["Rows"][1:]:
            data.append([col.get("VarCharValue", None) for col in row["Data"]])
        next_token = result_response.get("NextToken")
        if not next_token:
            break 
    df = pd.DataFrame(data, columns=columns)
    df['año'] = pd.to_datetime(df['año'].astype(str))
    df['total general'] = df['total general'].astype(int) 

    #Estandarizar Municipios
    loc_0 = df.columns.get_loc('municipio')
    df_split = df['municipio'].str.split(pat='^\\d+\\s+', expand=True).add_prefix('municipio_')
    df = pd.concat([df.iloc[:, :loc_0], df_split, df.iloc[:, loc_0:]], axis=1)
    df = df.drop(columns=['municipio', 'municipio_0'])
    df = df.rename(columns={'municipio_1': 'municipio', 'total general' : 'nacimientos'})
    df['municipio'] = df['municipio'].str.lower()

    #Información Agrupada
    df = df.groupby(['departamento', 'municipio', 'año'])['nacimientos'].sum().reset_index()
    df = df.groupby(['departamento', 'año'])['nacimientos'].sum().reset_index()

Regresión

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
departamentos = df['departamento'].unique()
ultimo_anio_conocido = df['año'].dt.year
ultimo_anio_conocido = ultimo_anio_conocido.max()
anios_futuros = np.arange(ultimo_anio_conocido + 1, ultimo_anio_conocido + 1 + n_periodos_a_predecir).reshape(-1, 1)
for departamento in departamentos:
    df_departamento = df[df['departamento'] == departamento].copy()
    df_departamento["año_num"] = df_departamento['año'].dt.year
    X_hist = df_departamento[['año_num']]
    if len(X_hist) > 2:
        y_hist = df_departamento['nacimientos']
        model = LinearRegression()
        model.fit(X_hist, y_hist)
        if df_departamento['año_num'].max() == ultimo_anio_conocido:
            predicciones_futuras = model.predict(anios_futuros)
            # Imprimir las predicciones
            for anio, pred in zip(anios_futuros.flatten(), predicciones_futuras):
                df_resultados = pd.concat([df_resultados, pd.DataFrame({'departamento': [departamento], 'año': [anio], 'nacimientos': [int(pred)]})], ignore_index=True)

In [ ]:
import seaborn as sns
df_plot_pred = df_resultados.groupby(['año']).agg(totalgeneral_sum=('nacimientos', 'sum')).reset_index()
plt.figure(figsize=(12, 6))
sns.lineplot(x='año', y='totalgeneral_sum', data=df_plot_pred)
plt.title('Distribución de la variable total_general por año')
plt.xlabel('Año')
plt.ylabel('Total General')
plt.xticks(ticks=anios_futuros.flatten())
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Sarimax

In [ ]:
departamentos = df['departamento'].unique()
lista_de_resultados = []
df_real = pd.DataFrame()
ultimo_anio_conocido = df['año'].max()
for departamento in departamentos:
    df_departamento = df[df['departamento'] == departamento].copy()
    if df_departamento['año'].max() == ultimo_anio_conocido:
        if len(df_departamento) < 10: # Umbral mínimo de puntos de datos
            print(f"Datos insuficientes para {departamento}. Se omite el modelo.")
            continue # Salta a la siguiente iteración del bucle
        print(df_departamento)
        df_departamento = df_departamento.set_index('año')
        df_departamento.index.freq = 'YS'
        y = df_departamento['nacimientos']
        X = pd.DataFrame(index=y.index)
        X['Pandemia'] = 0
        X.loc['2020-01-01':'2022-01-01', 'Pandemia'] = 1
        print(f"--- Entrenando modelo ARIMAX para el Departamento: {departamento} ---")
        arimax_model = pm.auto_arima(y, X=X,
                                    seasonal=False,
                                    trace=True, # Muestra el proceso de búsqueda
                                    error_action='ignore',
                                    suppress_warnings=True)
        # Para el futuro, la variable 'Pandemia' vuelve a ser 0
        fig = arimax_model.plot_diagnostics(figsize=(12, 8))
        # Añadir un título general a toda la figura para más claridad
        fig.suptitle(f'Diagnóstico del Modelo para {departamento}', fontsize=16, y=1.02)
        plt.tight_layout()
        plt.show()
        X_futuro = pd.DataFrame({
            'Pandemia': [0] * n_periodos_a_predecir
        }, index=pd.date_range(start=ultimo_anio_conocido, periods=n_periodos_a_predecir+1, freq='YS')[1:])
        prediccion, conf_int = arimax_model.predict(n_periods=n_periodos_a_predecir, X=X_futuro, return_conf_int=True)
        print(f"\n--- PREDICCIÓN MODELO {departamento}---")
        print(prediccion)

        df_real = pd.DataFrame({
            'departamento': departamento,
            'año': y.index,
            'tipo': 'Real',
            'nacimientos': y.values,
            'intervalo_inferior': np.nan, # No aplica para datos reales
            'intervalo_superior': np.nan, # No aplica para datos reales
            'modelo_arima': str(arimax_model.order)
        })
        df_pred = pd.DataFrame({
            'departamento': departamento,
            'año': prediccion.index,
            'tipo': 'Predicción',
            'nacimientos': prediccion.values,
            'intervalo_inferior': conf_int[:, 0],
            'intervalo_superior': conf_int[:, 1],
            'modelo_arima': str(arimax_model.order)
        })

        df_completo_departamento = pd.concat([df_real, df_pred], ignore_index=True)
        
        lista_de_resultados.append(df_completo_departamento)

df_final_arima = pd.concat(lista_de_resultados, ignore_index=True)


   departamento        año  nacimientos
0      Amazonas 2008-01-01         1475
1      Amazonas 2009-01-01         1336
2      Amazonas 2010-01-01         1313
3      Amazonas 2011-01-01         1420
4      Amazonas 2012-01-01         1603
5      Amazonas 2013-01-01         1420
6      Amazonas 2014-01-01         1550
7      Amazonas 2015-01-01         1491
8      Amazonas 2016-01-01         1340
9      Amazonas 2017-01-01         1433
10     Amazonas 2018-01-01         1382
11     Amazonas 2019-01-01         1175
12     Amazonas 2020-01-01         1127
13     Amazonas 2021-01-01         1168
14     Amazonas 2022-01-01         1117
15     Amazonas 2023-01-01          938
--- Entrenando modelo ARIMAX para el Departamento: Amazonas ---
Performing stepwise search to minimize aic


c:\Users\USUARIO\OneDrive\Documentos\Education\Maestria_UNIR_BigData\TFM\Desarrollo\.venv\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\USUARIO\OneDrive\Documentos\Education\Maestria_UNIR_BigData\TFM\Desarrollo\.venv\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\USUARIO\OneDrive\Documentos\Education\Maestria_UNIR_BigData\TFM\Desarrollo\.venv\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\USUARIO\OneDrive\Documentos\Education\Maestria_UNIR_BigData\TFM\Desarrollo\.venv\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' 

 ARIMA(2,0,2)(0,0,0)[0]             : AIC=inf, Time=0.45 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=277.553, Time=0.02 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=inf, Time=0.03 sec


c:\Users\USUARIO\OneDrive\Documentos\Education\Maestria_UNIR_BigData\TFM\Desarrollo\.venv\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\USUARIO\OneDrive\Documentos\Education\Maestria_UNIR_BigData\TFM\Desarrollo\.venv\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\USUARIO\OneDrive\Documentos\Education\Maestria_UNIR_BigData\TFM\Desarrollo\.venv\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(0,0,1)(0,0,0)[0]             : AIC=inf, Time=0.17 sec


...

 ARIMA(2,0,0)(0,0,0)[0]             : AIC=inf, Time=0.15 sec


c:\Users\USUARIO\OneDrive\Documentos\Education\Maestria_UNIR_BigData\TFM\Desarrollo\.venv\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=209.981, Time=0.22 sec


c:\Users\USUARIO\OneDrive\Documentos\Education\Maestria_UNIR_BigData\TFM\Desarrollo\.venv\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
df_final_arima.to_csv('resultados_arima.csv', index=False)

Analisis de Resultados

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

df_total_general = df_final_arima.groupby(['año', 'tipo'])['nacimientos'].sum().reset_index()

df_total_real = df_total_general[df_total_general['tipo'] == 'Real']
df_total_pred = df_total_general[df_total_general['tipo'] == 'Predicción']

ultimo_punto_real = df_total_real.iloc[[-1]] 
primer_punto_pred = df_total_pred.iloc[[0]]
df_conexion = pd.concat([ultimo_punto_real, primer_punto_pred], ignore_index=True)

plt.figure(figsize=(14, 7))

plt.plot(df_total_real['año'], df_total_real['nacimientos'], 
         label='Total Real Histórico', color='royalblue', linewidth=2, marker='.')

plt.plot(df_conexion['año'], df_conexion['nacimientos'], 
         color='darkorange', linestyle='--', linewidth=2)

plt.plot(df_total_pred['año'], df_total_pred['nacimientos'], 
         label='Total Predicho Futuro (Suma de Modelos)', color='darkorange', linestyle='--', marker='o')
todos_los_anios = pd.concat([df_total_real['año'], df_total_pred['año']]).unique()
etiquetas_de_anio = [d.year for d in todos_los_anios]
plt.xticks(ticks=todos_los_anios, labels=etiquetas_de_anio, rotation=45)


# 6. Añadir títulos y etiquetas para claridad
plt.title('Tendencia General de Nacimientos (Real vs. Predicción)', fontsize=16)
plt.xlabel('Año', fontsize=12)
plt.ylabel('Número Total de Nacimientos', fontsize=12)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

# Formatear el eje Y
ax = plt.gca()
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, p: format(int(x), ',')))

plt.tight_layout()
plt.show()
